In [2]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime

import backtrader as bt

In [3]:
class TestSizer(bt.Sizer):
    params = dict(stake=1)

    def _getsizing(self, comminfo, cash, data, isbuy):
        dt, i = self.strategy.datetime.date(), data._id
        s = self.p.stake * (1 + (not isbuy))
        print('{} Data {} OType {} Sizing to {}'.format(
            dt, data._name, ('buy' * isbuy) or 'sell', s))

        return s

In [12]:
class St(bt.Strategy):
    params = dict(
        enter=[1, 3, 4],  # data ids are 1 based
        hold=[7, 10, 15],  # data ids are 1 based
        usebracket=True,
        rawbracket=True,
        pentry=0.015,
        plimits=0.03,
        valid=10,
    )

    def notify_order(self, order):
        if order.status == order.Submitted:
            return

        dt, dn = self.datetime.date(), order.data._name
        print('{} {} Order {} Status {}'.format(
            dt, dn, order.ref, order.getstatusname())
        )

        whichord = ['main', 'stop', 'limit', 'close']
        if not order.alive():  # not alive - nullify
            dorders = self.o[order.data]
            idx = dorders.index(order)
            dorders[idx] = None
            print('-- No longer alive {} Ref'.format(whichord[idx]))

            if all(x is None for x in dorders):
                dorders[:] = []  # empty list - New orders allowed

    def __init__(self):
        self.o = dict()  # orders per data (main, stop, limit, manual-close)
        self.holding = dict()  # holding periods per data

    def next(self):
        for i, d in enumerate(self.datas):
            dt, dn = self.datetime.date(), d._name
            pos = self.getposition(d).size
            print('{} {} Position {}'.format(dt, dn, pos))

            if not pos and not self.o.get(d, None):  # no market / no orders
                if dt.weekday() == self.p.enter[i]:
                    if not self.p.usebracket:
                        self.o[d] = [self.buy(data=d)]
                        print('{} {} Buy {}'.format(dt, dn, self.o[d][0].ref))

                    else:
                        p = d.close[0] * (1.0 - self.p.pentry)
                        pstp = p * (1.0 - self.p.plimits)
                        plmt = p * (1.0 + self.p.plimits)
                        valid = datetime.timedelta(self.p.valid)

                        if self.p.rawbracket:
                            o1 = self.buy(data=d, exectype=bt.Order.Limit,
                                          price=p, valid=valid, transmit=False)

                            o2 = self.sell(data=d, exectype=bt.Order.Stop,
                                           price=pstp, size=o1.size,
                                           transmit=False, parent=o1)

                            o3 = self.sell(data=d, exectype=bt.Order.Limit,
                                           price=plmt, size=o1.size,
                                           transmit=True, parent=o1)

                            self.o[d] = [o1, o2, o3]

                        else:
                            self.o[d] = self.buy_bracket(
                                data=d, price=p, stopprice=pstp,
                                limitprice=plmt, oargs=dict(valid=valid))

                        print('{} {} Main {} Stp {} Lmt {}'.format(
                            dt, dn, *(x.ref for x in self.o[d])))

                    self.holding[d] = 0

            elif pos:  # exiting can also happen after a number of days
                self.holding[d] += 1
                if self.holding[d] >= self.p.hold[i]:
                    o = self.close(data=d)
                    self.o[d].append(o)  # manual order to list of orders
                    print('{} {} Manual Close {}'.format(dt, dn, o.ref))
                    if self.p.usebracket:
                        self.cancel(self.o[d][1])  # cancel stop side
                        print('{} {} Cancel {}'.format(dt, dn, self.o[d][1]))

In [22]:
if __name__ == '__main__':
    
    cerebro = bt.Cerebro()
    folder_path = '/home/aviral/backtrader-master/datas/'
    data0_path = folder_path + 'orcl-1995-2014.txt'
    data1_path = folder_path + 'nvda-1999-2014.txt'
    data2_path = folder_path + 'yhoo-1996-2014.txt'

    fdate = datetime.datetime(2001,1,1)
    todate = datetime.datetime(2007, 1, 1)
    # Data feed
    data0 = bt.feeds.YahooFinanceCSVData(dataname=data0_path, fromdate=fdate, todate=todate)
    cerebro.adddata(data0, name='d0')

    data1 = bt.feeds.YahooFinanceCSVData(dataname=data1_path, fromdate=fdate, todate=todate)
    data1.plotinfo.plotmaster = data0
    cerebro.adddata(data1, name='d1')

    data2 = bt.feeds.YahooFinanceCSVData(dataname=data2_path, fromdate=fdate, todate=todate)
    data2.plotinfo.plotmaster = data0
    cerebro.adddata(data2, name='d2')

    # Broker
    cerebro.broker.setcommission(commission=0.001)

    # Sizer
    cerebro.addsizer(TestSizer)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')

    # Strategy
    cerebro.addstrategy(St)

    # Execute
    thestrat = cerebro.run()

2001-01-02 d0 Position 0
2001-01-02 Data d0 OType buy Sizing to 1
2001-01-02 d0 Main 6115 Stp 6116 Lmt 6117
2001-01-02 d1 Position 0
2001-01-02 d2 Position 0
2001-01-03 d0 Order 6115 Status Accepted
2001-01-03 d0 Order 6116 Status Accepted
2001-01-03 d0 Order 6117 Status Accepted
2001-01-03 d0 Order 6115 Status Completed
-- No longer alive main Ref
2001-01-03 d0 Position 1
2001-01-03 d1 Position 0
2001-01-03 d2 Position 0
2001-01-04 d0 Order 6117 Status Completed
-- No longer alive limit Ref
2001-01-04 d0 Order 6116 Status Canceled
-- No longer alive stop Ref
2001-01-04 d0 Position 0
2001-01-04 d1 Position 0
2001-01-04 Data d1 OType buy Sizing to 1
2001-01-04 d1 Main 6118 Stp 6119 Lmt 6120
2001-01-04 d2 Position 0
2001-01-05 d1 Order 6118 Status Accepted
2001-01-05 d1 Order 6119 Status Accepted
2001-01-05 d1 Order 6120 Status Accepted
2001-01-05 d1 Order 6118 Status Completed
-- No longer alive main Ref
2001-01-05 d0 Position 0
2001-01-05 d1 Position 1
2001-01-05 d2 Position 0
2001-01-

2001-08-16 d1 Position 0
2001-08-16 Data d1 OType buy Sizing to 1
2001-08-16 d1 Main 6361 Stp 6362 Lmt 6363
2001-08-16 d2 Position 0
2001-08-17 d1 Order 6361 Status Accepted
2001-08-17 d1 Order 6362 Status Accepted
2001-08-17 d1 Order 6363 Status Accepted
2001-08-17 d1 Order 6361 Status Completed
-- No longer alive main Ref
2001-08-17 d0 Position 0
2001-08-17 d1 Position 1
2001-08-17 d2 Position 0
2001-08-17 Data d2 OType buy Sizing to 1
2001-08-17 d2 Main 6364 Stp 6365 Lmt 6366
2001-08-20 d2 Order 6364 Status Accepted
2001-08-20 d2 Order 6365 Status Accepted
2001-08-20 d2 Order 6366 Status Accepted
2001-08-20 d1 Order 6362 Status Completed
-- No longer alive stop Ref
2001-08-20 d1 Order 6363 Status Canceled
-- No longer alive limit Ref
2001-08-20 d2 Order 6364 Status Completed
-- No longer alive main Ref
2001-08-20 d0 Position 0
2001-08-20 d1 Position 0
2001-08-20 d2 Position 1
2001-08-21 d2 Order 6365 Status Completed
-- No longer alive stop Ref
2001-08-21 d2 Order 6366 Status Cancel

2002-02-12 d0 Main 6538 Stp 6539 Lmt 6540
2002-02-12 d1 Position 0
2002-02-12 d2 Position 1
2002-02-13 d0 Order 6538 Status Accepted
2002-02-13 d0 Order 6539 Status Accepted
2002-02-13 d0 Order 6540 Status Accepted
2002-02-13 d2 Order 6537 Status Completed
-- No longer alive limit Ref
2002-02-13 d2 Order 6536 Status Canceled
-- No longer alive stop Ref
2002-02-13 d0 Position 0
2002-02-13 d1 Position 0
2002-02-13 d2 Position 0
2002-02-14 d0 Position 0
2002-02-14 d1 Position 0
2002-02-14 Data d1 OType buy Sizing to 1
2002-02-14 d1 Main 6541 Stp 6542 Lmt 6543
2002-02-14 d2 Position 0
2002-02-15 d1 Order 6541 Status Accepted
2002-02-15 d1 Order 6542 Status Accepted
2002-02-15 d1 Order 6543 Status Accepted
2002-02-15 d0 Order 6538 Status Completed
-- No longer alive main Ref
2002-02-15 d1 Order 6541 Status Completed
-- No longer alive main Ref
2002-02-15 d0 Position 1
2002-02-15 d1 Position 1
2002-02-15 d2 Position 0
2002-02-15 Data d2 OType buy Sizing to 1
2002-02-15 d2 Main 6544 Stp 6545 

2002-08-30 d1 Order 6766 Status Accepted
2002-08-30 d1 Order 6767 Status Accepted
2002-08-30 d1 Order 6768 Status Accepted
2002-08-30 d1 Order 6766 Status Completed
-- No longer alive main Ref
2002-08-30 d0 Position 0
2002-08-30 d1 Position 1
2002-08-30 d2 Position 0
2002-08-30 Data d2 OType buy Sizing to 1
2002-08-30 d2 Main 6769 Stp 6770 Lmt 6771
2002-09-03 d2 Order 6769 Status Accepted
2002-09-03 d2 Order 6770 Status Accepted
2002-09-03 d2 Order 6771 Status Accepted
2002-09-03 d1 Order 6767 Status Completed
-- No longer alive stop Ref
2002-09-03 d1 Order 6768 Status Canceled
-- No longer alive limit Ref
2002-09-03 d2 Order 6769 Status Completed
-- No longer alive main Ref
2002-09-03 d0 Position 0
2002-09-03 Data d0 OType buy Sizing to 1
2002-09-03 d0 Main 6772 Stp 6773 Lmt 6774
2002-09-03 d1 Position 0
2002-09-03 d2 Position 1
2002-09-04 d0 Order 6772 Status Accepted
2002-09-04 d0 Order 6773 Status Accepted
2002-09-04 d0 Order 6774 Status Accepted
2002-09-04 d2 Order 6770 Status Com

2003-03-21 d2 Main 6988 Stp 6989 Lmt 6990
2003-03-24 d2 Order 6988 Status Accepted
2003-03-24 d2 Order 6989 Status Accepted
2003-03-24 d2 Order 6990 Status Accepted
2003-03-24 d0 Order 6976 Status Expired
-- No longer alive main Ref
2003-03-24 d0 Order 6977 Status Canceled
-- No longer alive stop Ref
2003-03-24 d0 Order 6978 Status Canceled
-- No longer alive limit Ref
2003-03-24 d1 Order 6986 Status Completed
-- No longer alive stop Ref
2003-03-24 d1 Order 6987 Status Canceled
-- No longer alive limit Ref
2003-03-24 d2 Order 6988 Status Completed
-- No longer alive main Ref
2003-03-24 d0 Position 0
2003-03-24 d1 Position 0
2003-03-24 d2 Position 1
2003-03-25 d2 Order 6990 Status Completed
-- No longer alive limit Ref
2003-03-25 d2 Order 6989 Status Canceled
-- No longer alive stop Ref
2003-03-25 d0 Position 0
2003-03-25 Data d0 OType buy Sizing to 1
2003-03-25 d0 Main 6991 Stp 6992 Lmt 6993
2003-03-25 d1 Position 0
2003-03-25 d2 Position 0
2003-03-26 d0 Order 6991 Status Accepted
2003

2003-11-14 d2 Main 7202 Stp 7203 Lmt 7204
2003-11-17 d2 Order 7202 Status Accepted
2003-11-17 d2 Order 7203 Status Accepted
2003-11-17 d2 Order 7204 Status Accepted
2003-11-17 d0 Order 7193 Status Expired
-- No longer alive main Ref
2003-11-17 d0 Order 7194 Status Canceled
-- No longer alive stop Ref
2003-11-17 d0 Order 7195 Status Canceled
-- No longer alive limit Ref
2003-11-17 d1 Order 7196 Status Expired
-- No longer alive main Ref
2003-11-17 d1 Order 7197 Status Canceled
-- No longer alive stop Ref
2003-11-17 d1 Order 7198 Status Canceled
-- No longer alive limit Ref
2003-11-17 d2 Order 7202 Status Completed
-- No longer alive main Ref
2003-11-17 d0 Position 0
2003-11-17 d1 Position 0
2003-11-17 d2 Position 1
2003-11-18 d2 Order 7203 Status Completed
-- No longer alive stop Ref
2003-11-18 d2 Order 7204 Status Canceled
-- No longer alive limit Ref
2003-11-18 d0 Position 0
2003-11-18 Data d0 OType buy Sizing to 1
2003-11-18 d0 Main 7205 Stp 7206 Lmt 7207
2003-11-18 d1 Position 0
200

2004-07-30 d1 Order 7430 Status Accepted
2004-07-30 d1 Order 7431 Status Accepted
2004-07-30 d1 Order 7432 Status Accepted
2004-07-30 d0 Position 1
2004-07-30 d1 Position 0
2004-07-30 d2 Position 0
2004-07-30 Data d2 OType buy Sizing to 1
2004-07-30 d2 Main 7433 Stp 7434 Lmt 7435
2004-08-02 d2 Order 7433 Status Accepted
2004-08-02 d2 Order 7434 Status Accepted
2004-08-02 d2 Order 7435 Status Accepted
2004-08-02 d0 Order 7429 Status Completed
-- No longer alive limit Ref
2004-08-02 d0 Order 7428 Status Canceled
-- No longer alive stop Ref
2004-08-02 d2 Order 7433 Status Completed
-- No longer alive main Ref
2004-08-02 d0 Position 0
2004-08-02 d1 Position 0
2004-08-02 d2 Position 1
2004-08-03 d1 Order 7430 Status Completed
-- No longer alive main Ref
2004-08-03 d2 Order 7434 Status Completed
-- No longer alive stop Ref
2004-08-03 d2 Order 7435 Status Canceled
-- No longer alive limit Ref
2004-08-03 d0 Position 0
2004-08-03 Data d0 OType buy Sizing to 1
2004-08-03 d0 Main 7436 Stp 7437 Lm

2005-04-13 d0 Order 7635 Status Completed
-- No longer alive main Ref
2005-04-13 d0 Position 1
2005-04-13 d1 Position 0
2005-04-13 d2 Position 1
2005-04-14 d0 Position 1
2005-04-14 d1 Position 0
2005-04-14 Data d1 OType buy Sizing to 1
2005-04-14 d1 Main 7641 Stp 7642 Lmt 7643
2005-04-14 d2 Position 1
2005-04-15 d1 Order 7641 Status Accepted
2005-04-15 d1 Order 7642 Status Accepted
2005-04-15 d1 Order 7643 Status Accepted
2005-04-15 d0 Order 7636 Status Completed
-- No longer alive stop Ref
2005-04-15 d0 Order 7637 Status Canceled
-- No longer alive limit Ref
2005-04-15 d2 Order 7639 Status Completed
-- No longer alive stop Ref
2005-04-15 d2 Order 7640 Status Canceled
-- No longer alive limit Ref
2005-04-15 d1 Order 7641 Status Completed
-- No longer alive main Ref
2005-04-15 d0 Position 0
2005-04-15 d1 Position 1
2005-04-15 d2 Position 0
2005-04-15 Data d2 OType buy Sizing to 1
2005-04-15 d2 Main 7644 Stp 7645 Lmt 7646
2005-04-18 d2 Order 7644 Status Accepted
2005-04-18 d2 Order 7645 

2005-12-21 d0 Order 7838 Status Accepted
2005-12-21 d0 Order 7839 Status Accepted
2005-12-21 d0 Order 7840 Status Accepted
2005-12-21 d2 Order 7836 Status Completed
-- No longer alive stop Ref
2005-12-21 d2 Order 7837 Status Canceled
-- No longer alive limit Ref
2005-12-21 d0 Position 0
2005-12-21 d1 Position 1
2005-12-21 d2 Position 0
2005-12-22 d1 Order 7834 Status Completed
-- No longer alive limit Ref
2005-12-22 d1 Order 7833 Status Canceled
-- No longer alive stop Ref
2005-12-22 d0 Position 0
2005-12-22 d1 Position 0
2005-12-22 Data d1 OType buy Sizing to 1
2005-12-22 d1 Main 7841 Stp 7842 Lmt 7843
2005-12-22 d2 Position 0
2005-12-23 d1 Order 7841 Status Accepted
2005-12-23 d1 Order 7842 Status Accepted
2005-12-23 d1 Order 7843 Status Accepted
2005-12-23 d0 Position 0
2005-12-23 d1 Position 0
2005-12-23 d2 Position 0
2005-12-23 Data d2 OType buy Sizing to 1
2005-12-23 d2 Main 7844 Stp 7845 Lmt 7846
2005-12-27 d2 Order 7844 Status Accepted
2005-12-27 d2 Order 7845 Status Accepted
2

2006-08-29 d1 Position 0
2006-08-29 d2 Position 1
2006-08-30 d0 Order 8049 Status Completed
-- No longer alive limit Ref
2006-08-30 d0 Order 8048 Status Canceled
-- No longer alive stop Ref
2006-08-30 d0 Position 0
2006-08-30 d1 Position 0
2006-08-30 d2 Position 1
2006-08-31 d0 Position 0
2006-08-31 d1 Position 0
2006-08-31 Data d1 OType buy Sizing to 1
2006-08-31 d1 Main 8053 Stp 8054 Lmt 8055
2006-08-31 d2 Position 1
2006-09-01 d1 Order 8053 Status Accepted
2006-09-01 d1 Order 8054 Status Accepted
2006-09-01 d1 Order 8055 Status Accepted
2006-09-01 d1 Order 8053 Status Completed
-- No longer alive main Ref
2006-09-01 d0 Position 0
2006-09-01 d1 Position 1
2006-09-01 d2 Position 1
2006-09-05 d1 Order 8054 Status Completed
-- No longer alive stop Ref
2006-09-05 d1 Order 8055 Status Canceled
-- No longer alive limit Ref
2006-09-05 d0 Position 0
2006-09-05 Data d0 OType buy Sizing to 1
2006-09-05 d0 Main 8056 Stp 8057 Lmt 8058
2006-09-05 d1 Position 0
2006-09-05 d2 Position 1
2006-09-06 

In [24]:
print('Sharpe Ratio:', thestrat[0].analyzers.sharpe.get_analysis())

Sharpe Ratio: OrderedDict([('sharperatio', -19.629457408056087)])
